# Random Forest Model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import helper_functions as hf
import Random_Forest_Model as rf
import dataframe_image as dfi
from itertools import product

In [4]:
df = hf.load_data()
df = hf.create_engineered_plus_fundamental_features(df)
df = hf.create_binary_labels(df,[1,20,60])

raw = ['ret','volume']
eng = [
    'mean_20', 'mean_60',
    'vol_20', 'vol_60',
    'mom_20', 'mom_60',
    'ema_12', 'ema_26', 'ema_cross',
    'skew_20', 'skew_60',
    'kurt_20', 'kurt_60',
    'vol_z'
]
fund = [
    'eps',
    'profit_margin',
    'revenue_growth',
    'income_growth',
    'gross_margin',
    'operating_margin',
    'sga_ratio',
    'rd_ratio',
    'cost_ratio',
    'net_income_per_share',
    'tax_burden',
    'nonop_ratio',
    'abnormal_ratio',
    'revenue_per_share',
    'da_ratio',
    'interest_coverage',
    'interest_burden'
]
fund_eng = fund + eng 

#rt = df.pivot(index='date', columns='ticker', values='ret')


/Users/evi/Desktop/ML-Stock-Predictor/helper_functions.py:64: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['revenue_growth'] = df.groupby('ticker')['revenue'].pct_change()
/Users/evi/Desktop/ML-Stock-Predictor/helper_functions.py:66: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["income_growth"] = df.groupby("ticker")["net_income"].pct_change()


In [5]:
df

,index_x,ticker,simfinid_x,date,open,high,low,close,adj_close,volume,...,tax_burden,nonop_ratio,abnormal_ratio,revenue_per_share,da_ratio,interest_coverage,interest_burden,y_1,y_20,y_60
0,16033,AAPL,111052,2019-12-23,70.13,71.06,70.09,71.00,68.53,98711532,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0
1,40152,ABT,63877,2019-12-23,86.31,87.46,86.28,87.35,78.56,4067769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0
2,95854,ADBE,14099,2019-12-23,328.83,329.88,327.26,328.95,328.95,2210706,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0
3,296720,AMGN,65735,2019-12-23,243.98,244.00,241.59,243.03,203.47,1686220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,333254,AMZN,62747,2019-12-23,89.41,89.65,89.23,89.65,89.65,42749860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60755,5695307,UNH,367714,2024-11-25,597.58,609.84,595.01,605.83,592.89,5146485,...,0.793162,-0.010653,-0.000198,108.408602,-0.010325,-8.108007,0.876665,NaN,NaN,NaN
60756,5755996,V,107326,2024-11-25,311.86,313.66,309.86,313.19,310.99,14189302,...,0.806980,0.002392,-0.022668,4.612470,-0.028491,-37.312500,1.003502,NaN,NaN,NaN
60757,5926877,VZ,101219,2024-11-25,43.34,44.05,43.31,43.98,41.15,28365883,...,0.768480,-0.048725,NaN,7.888757,-0.133753,-3.544258,0.725953,NaN,NaN,NaN
60758,6024283,WMT,239962,2024-11-25,90.50,90.95,89.06,89.50,88.64,25078633,...,0.734671,-0.002819,-0.000778,20.983420,NaN,-14.033473,0.928742,NaN,NaN,NaN


## Run all the models with variations in Features and Horizons

In [6]:
features_map = {
    "raw": raw,
    "eng": eng,
    "fund": fund,
    "fund_eng": fund_eng
}

In [7]:
features_sets = ['raw', 'eng', 'fund', 'fund_eng']
targets = ['y_1', 'y_20', 'y_60']
#features_sets = ['raw']
#targets = ['y_1']
res = []

for f, t in product(features_sets, targets):
    print(f, t, type(f), type(t))
    features = features_map[f]
    # create features
    data = hf.prune(df, features, t)

    train, val, test = hf.time_split(data)
    print(f, t)
    ## not returning accuracy right now
    ### FIX
    #val_acc, test_acc = rf.run_optimize_eval_RF(train, val, test, features, t)
    val_auc, val_acc, test_auc, test_acc = rf.run_optimize_eval_RF(train, val, test, features, t)


    res.append({
        "Horizon": t,
        "Features": f,
        "Val AUC": val_auc,
        "Val Accuracy": val_acc,
        "Test AUC": test_auc,
        "Test Accuracy": test_acc
    })


res

raw y_1 <class 'str'> <class 'str'>
raw y_1
              precision    recall  f1-score   support

           0       0.47      0.41      0.44      4232
           1       0.53      0.59      0.56      4833

    accuracy                           0.51      9065
   macro avg       0.50      0.50      0.50      9065
weighted avg       0.50      0.51      0.50      9065

OOB Score: 0.506366649223792

Top 10 Feature Importances:
ret       0.508936
volume    0.491064
dtype: float64
raw y_20 <class 'str'> <class 'str'>
raw y_20
              precision    recall  f1-score   support

           0       0.40      0.74      0.52      3480
           1       0.63      0.28      0.39      5438

    accuracy                           0.46      8918
   macro avg       0.51      0.51      0.45      8918
weighted avg       0.54      0.46      0.44      8918

OOB Score: 0.5000098400015744

Top 10 Feature Importances:
volume    0.508437
ret       0.491563
dtype: float64
raw y_60 <class 'str'> <class 'st

[{'Horizon': 'y_1',
  'Features': 'raw',
  'Val AUC': 0.5053005990896623,
  'Val Accuracy': 0.5061443932411674,
  'Test AUC': 0.498329214673693,
  'Test Accuracy': 0.505019305019305},
 {'Horizon': 'y_20',
  'Features': 'raw',
  'Val AUC': 0.5133946082968286,
  'Val Accuracy': 0.46180439388870304,
  'Test AUC': 0.5076126703106704,
  'Test Accuracy': 0.4610899304776856},
 {'Horizon': 'y_60',
  'Features': 'raw',
  'Val AUC': 0.5150323667352983,
  'Val Accuracy': 0.5043237634036666,
  'Test AUC': 0.5138342882048625,
  'Test Accuracy': 0.4902597402597403},
 {'Horizon': 'y_1',
  'Features': 'eng',
  'Val AUC': 0.49794541113376506,
  'Val Accuracy': 0.49613743802605786,
  'Test AUC': 0.4982897654001105,
  'Test Accuracy': 0.4967532467532468},
 {'Horizon': 'y_20',
  'Features': 'eng',
  'Val AUC': 0.4898147385872731,
  'Val Accuracy': 0.49132066619751347,
  'Test AUC': 0.47768973557855976,
  'Test Accuracy': 0.480712516220361},
 {'Horizon': 'y_60',
  'Features': 'eng',
  'Val AUC': 0.47879026

In [8]:
res_df = pd.DataFrame(res)
val_acc_df = res_df.pivot(index = "Features", columns = "Horizon", values = "Val Accuracy")

val_acc_df = val_acc_df.rename(columns={
    "y_1": "1-day horizon",
    "y_20": "20-day horizon",
    "y_60": "60-day horizon"
})

val_acc_df = val_acc_df.rename(index={
    "raw": "Raw",
    "eng": "Engineered",
    "fund": "Fundamental",
    "fund_eng": "Fund + Eng"
})

val_acc_st = (
    val_acc_df.style
      .set_caption("Table 1: Random Forest - Summary Validation Accuracy")
      .format("{:.4f}")              # numeric formatting
      .set_table_styles([
          {"selector": "table", "props": "width:100%; border-collapse:separate; border-spacing:10px;"},
          {"selector": "th, td", "props": "padding:10px;"},
          {"selector": "th", "props": "font-size:12pt;"},
          {"selector": "td", "props": "font-size:11pt;"},
      ])
      .set_properties(**{"min-width": "120px"})
)

#dfi.export(test_st, "random_forest_table_test.png", table_conversion="matplotlib")
dfi.export(val_acc_st, "random_forest_table_validation_accuracy.png", table_conversion="matplotlib")

In [9]:
val_acc_st

Horizon,1-day horizon,20-day horizon,60-day horizon
Features,,,
Engineered,0.4961,0.4913,0.5583
Fundamental,0.5222,0.5000,0.5943
Fund + Eng,0.5254,0.5655,0.6369
Raw,0.5061,0.4618,0.5043


In [10]:
res_df = pd.DataFrame(res)
val_auc_df = res_df.pivot(index = "Features", columns = "Horizon", values = "Val AUC")

val_auc_df = val_auc_df.rename(columns={
    "y_1": "1-day horizon",
    "y_20": "20-day horizon",
    "y_60": "60-day horizon"
})

val_auc_df = val_auc_df.rename(index={
    "raw": "Raw",
    "eng": "Engineered",
    "fund": "Fundamental",
    "fund_eng": "Fund + Eng"
})

val_auc_st = (
    val_auc_df.style
      .set_caption("Table 1: Random Forest - Summary Validation AUC")
      .format("{:.4f}")              # numeric formatting
      .set_table_styles([
          {"selector": "table", "props": "width:100%; border-collapse:separate; border-spacing:10px;"},
          {"selector": "th, td", "props": "padding:10px;"},
          {"selector": "th", "props": "font-size:12pt;"},
          {"selector": "td", "props": "font-size:11pt;"},
      ])
      .set_properties(**{"min-width": "120px"})
)

#dfi.export(test_st, "random_forest_table_test.png", table_conversion="matplotlib")
dfi.export(val_auc_st, "random_forest_table_validation_AUC.png", table_conversion="matplotlib")

In [11]:
val_auc_st

Horizon,1-day horizon,20-day horizon,60-day horizon
Features,,,
Engineered,0.4979,0.4898,0.4788
Fundamental,0.5312,0.5494,0.5346
Fund + Eng,0.5401,0.5797,0.6776
Raw,0.5053,0.5134,0.5150


In [12]:
test_acc_df = res_df.pivot(index = "Features", columns = "Horizon", values = "Test Accuracy")
test_acc_df = test_acc_df.rename(columns={
    "y_1": "1-day horizon",
    "y_20": "20-day horizon",
    "y_60": "60-day horizon"
})

test_acc_df = test_acc_df.rename(index={
    "raw": "Raw",
    "eng": "Engineered",
    "fund": "Fundamental",
    "fund_eng": "Fund + Eng"
})

test_acc_st = (
    test_acc_df.style
      .set_caption("Table 2: Random Forest - Summary Testing Accuracy")
      .format("{:.4f}")              # numeric formatting
      .set_table_styles([
          {"selector": "table", "props": "width:100%; border-collapse:separate; border-spacing:10px;"},
          {"selector": "th, td", "props": "padding:10px;"},
          {"selector": "th", "props": "font-size:12pt;"},
          {"selector": "td", "props": "font-size:11pt;"},
      ])
      .set_properties(**{"min-width": "120px"})
)
dfi.export(test_acc_st, "random_forest_table_test_accuracy.png", table_conversion="matplotlib")

In [13]:
test_acc_st

Horizon,1-day horizon,20-day horizon,60-day horizon
Features,,,
Engineered,0.4968,0.4807,0.5687
Fundamental,0.5130,0.5055,0.4852
Fund + Eng,0.4727,0.5260,0.5353
Raw,0.5050,0.4611,0.4903


In [14]:
test_auc_df = res_df.pivot(index = "Features", columns = "Horizon", values = "Test AUC")
test_auc_df = test_auc_df.rename(columns={
    "y_1": "1-day horizon",
    "y_20": "20-day horizon",
    "y_60": "60-day horizon"
})

test_auc_df = test_auc_df.rename(index={
    "raw": "Raw",
    "eng": "Engineered",
    "fund": "Fundamental",
    "fund_eng": "Fund + Eng"
})

test_auc_st = (
    test_auc_df.style
      .set_caption("Table 2: Random Forest - Summary Testing Accuracy")
      .format("{:.4f}")              # numeric formatting
      .set_table_styles([
          {"selector": "table", "props": "width:100%; border-collapse:separate; border-spacing:10px;"},
          {"selector": "th, td", "props": "padding:10px;"},
          {"selector": "th", "props": "font-size:12pt;"},
          {"selector": "td", "props": "font-size:11pt;"},
      ])
      .set_properties(**{"min-width": "120px"})
)
dfi.export(test_auc_st, "random_forest_table_test_AUC.png", table_conversion="matplotlib")

In [15]:
test_auc_st

Horizon,1-day horizon,20-day horizon,60-day horizon
Features,,,
Engineered,0.4983,0.4777,0.5023
Fundamental,0.5285,0.5489,0.4920
Fund + Eng,0.4963,0.5676,0.6403
Raw,0.4983,0.5076,0.5138
